In [3]:
import pandas as pd
import numpy as np
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification

In [46]:
data = pd.read_csv('/Users/kristian/Documents/Skole/7. semester/NLP/Exam/ASD_classification/CHI_data.csv')


In [57]:
def preprocess_tokens(df):

    # Remove punctuations and numbers
    df['tokens2'] = df['tokens2'].str.replace('[^a-zA-Z]', ' ', regex=True)

    # Single character removal
    df['tokens2'] = df['tokens2'].str.replace(r"\s+[a-zA-Z]\s+", ' ', regex=True)

    # Removing multiple spaces
    df['tokens2'] = df['tokens2'].str.replace(r'\s+', ' ', regex=True)

    # Drop spaces created when deleting single period values
    df = df[df.tokens2 != ' ']

    return df

# use function on data frame
data = preprocess_tokens(data)

In [58]:
data.head(20)

,Unnamed: 0,participant,age,id,group,tokens2
6,110,CHI,45.633333,11312/a-00032743-1,TD,cow
12,135,CHI,45.633333,11312/a-00032743-1,TD,tree
13,141,CHI,45.633333,11312/a-00032743-1,TD,bandaid
14,147,CHI,45.633333,11312/a-00032743-1,TD,ow ipep
17,162,CHI,45.633333,11312/a-00032743-1,TD,brush
18,166,CHI,45.633333,11312/a-00032743-1,TD,comb
20,170,CHI,45.633333,11312/a-00032743-1,TD,hair
23,185,CHI,45.633333,11312/a-00032743-1,TD,a
26,192,CHI,45.633333,11312/a-00032743-1,TD,bear
28,202,CHI,45.633333,11312/a-00032743-1,TD,on the floor


In [9]:
model_checkpoint = "distilbert-base-uncased"

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]